In [3]:
import sys; sys.path.append('..')
import warnings
warnings.filterwarnings('ignore')
from multiprompt import *

In [4]:
def get_df_poems(min_lines=8, fn='/Users/rj416/Dropbox/Prof/Data/data.allpoems.pkl'):
    df_poems = pd.read_pickle(fn).fillna('')
    
    excl_prompts=[
        'Write an unryhmed poem in the style of Shakespeare\'s dramatic monologues.',
        'Write a poem in the style of Shakespeare\'s dramatic monologues.',
        'Write a poem in the style of e.e. cummings',
        # 'Write a poem in the style of Walt Whitman.',
        'Write a poem in the style of Wallace Stevens.',
        'Continue the following poem:\n\nTyping, typing, fingers on the keyboard\nThe keys crack and bend under sweat and weight,\n'
    ]

    df_poems = df_poems[~df_poems.prompt.isin(excl_prompts)]
    df_poems['num_lines'] = pd.to_numeric(df_poems['num_lines'], errors='coerce')
    return df_poems.query(f'num_lines >= {min_lines}')

In [5]:
pd.options.display.max_rows=1000
df = get_df_poems()
df = df[~df.model.str.startswith('b.')]

In [6]:
def generate_many(models, prompts, n=100, agent_name='rhyme4'):
    agent = Agent(agent_name)
    done = Counter((d['prompt']['model'], d['prompt']['user_prompt']) for d in agent.responses)
    for model,prompt in zip(df.model, df.prompt):
        done[(model,prompt)]+=1
    todo = []
    for m in models:
        for p in prompts:
            nnow = n - done.get((m,p), 0)
            for i in range(nnow):
                todo.append((m,p))
    # random.shuffle(todo)

    def save():
        agent = Agent('rhyme4')
        with open('/Users/rj416/Dropbox/Prof/Data/data.newpoems2.json', 'w') as of:
            json.dump(agent.responses, of)

    for i,(m,p) in enumerate(tqdm(todo)):
        agent = Agent(agent_name, model=m)
        agent.generate(p, _force=True, verbose=False)
        if not i%100: save()
    save()
    
    return agent.responses


In [7]:
models = df.model.unique().tolist()
prompts = df.prompt.unique().tolist()
models = [
#  'gpt-3.5-turbo',
#  'claude-3-opus-20240229',
#  'claude-3-haiku-20240307',
#  'claude-3-sonnet-20240229',
#  'gemini-pro',
#  'ollama/llama2-uncensored:latest',
'ollama/olmo2',
'ollama/olmo2:13b',
'ollama/llama3.1:8b',
'ollama/llama3.1:70b',
'ollama/llama2-uncensored:latest',
'ollama/deepseek-r1:8b',
'ollama/darkmoon/olmo:7B-instruct-q6-k',
#  'gpt-4-turbo'
 ]

In [8]:
generate_many(models, prompts)

  0%|          | 0/11249 [00:00<?, ?it/s]

ERROR:multiprompt.imports:Error in generate_async: litellm.APIConnectionError: [Errno 24] Too many open files
Traceback (most recent call last):
  File "/Users/rj416/github/multiprompt/venv/lib/python3.12/site-packages/litellm/main.py", line 470, in acompletion
    response = await init_response
               ^^^^^^^^^^^^^^^^^^^
  File "/Users/rj416/github/multiprompt/venv/lib/python3.12/site-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 478, in acompletion_stream_function
    completion_stream, _response_headers = await self.make_async_call_stream_helper(
                                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rj416/github/multiprompt/venv/lib/python3.12/site-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 519, in make_async_call_stream_helper
    async_httpx_client = get_async_httpx_client(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rj416/github/multiprompt/venv/lib/python3.12/site-package


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



APIConnectionError: litellm.APIConnectionError: [Errno 24] Too many open files
Traceback (most recent call last):
  File "/Users/rj416/github/multiprompt/venv/lib/python3.12/site-packages/litellm/main.py", line 470, in acompletion
    response = await init_response
               ^^^^^^^^^^^^^^^^^^^
  File "/Users/rj416/github/multiprompt/venv/lib/python3.12/site-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 478, in acompletion_stream_function
    completion_stream, _response_headers = await self.make_async_call_stream_helper(
                                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rj416/github/multiprompt/venv/lib/python3.12/site-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 519, in make_async_call_stream_helper
    async_httpx_client = get_async_httpx_client(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rj416/github/multiprompt/venv/lib/python3.12/site-packages/litellm/llms/custom_httpx/htt